<a href="https://colab.research.google.com/github/rajitakolla/computational_methods/blob/master/Biterm_model_for_user_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:12 https://cloud.r-project.org/b

In [1]:
# Write your code here
import urllib.request
import requests 
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)

In [2]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [3]:
def fetch_data(driver, class_name_val, css_selector_val):
  result = list()
  values = driver.find_elements_by_class_name(class_name_val)
  for value in values:
    result.append(value.find_element_by_css_selector(css_selector_val).text)
  return result
  pass

In [4]:
import time
driver.get("https://www.imdb.com/title/tt7286456/reviews?sort=userRating&dir=desc&ratingFilter=0")
#activate the button load_more and run the loop 4 times so to make sure we have 100 words
i=0
while(i<3):
  button = driver.find_element_by_xpath('//*[@id ="load-more-trigger"]').click()
  i = i+1
  time.sleep(2)

reviews = fetch_data(driver, "content", ".text.show-more__control")
user_names = fetch_data(driver, "display-name-date","a")

In [5]:

#create a data frame of usernames and reviews
required_df = pd.concat([pd.DataFrame(user_names,columns=['User Names']),pd.DataFrame(reviews,columns=['Reviews'])],axis=1)
required_df

,User Names,Reviews
0,morino-59166,When I saw the magnificent Heath ledger a few ...
1,asassandroo,I can say that i'm really grateful that movies...
2,samanehdds,I'm not a fan of Batman movies but went to see...
3,guillevica,"What a master piece, it so so wondeful, all in..."
4,ahurworth,Has to be the best movie I've seen in years. I...
...,...,...
95,shinginharig,Absolutely great performance. Phoenix owns thi...
96,afnansaeed-76293,One of the best movies. I really recommend it ...
97,legorocks-30147,
98,tusharshanker-96210,"This movie is truly a masterpiece. The acting,..."


In [6]:
import string
required_df["Punctuations"]=required_df['Reviews'].apply(lambda x:''.join([i for i in x if i not in string.punctuation]))
required_df["spl chars"] = required_df["Punctuations"].apply(lambda x:''.join([i for i in x if i.isalnum() or i == " "]))
required_df["numbers"] = required_df['spl chars'].apply(lambda x:''.join([i for i in x if i == " " or i.isalpha()]))

In [7]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
list_of_words = set(stopwords.words('english'))

In [9]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:

from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 
   
ps = PorterStemmer()

In [11]:
nltk.download('wordnet')
import textblob

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [19]:
required_df["stop_words"]=required_df['numbers'].apply(lambda x:' '.join([i for i in x.split() if i not in list_of_words]))
required_df["lower_case"]=required_df['stop_words'].apply(lambda x: x.lower())
required_df["tokenisation"]=required_df["lower_case"].apply(lambda x : word_tokenize(str(x)))
required_df["stemming"]=required_df["tokenisation"].apply(lambda x: ' '.join([ps.stem(i) for i in x]))
required_df["tokenisation1"]=required_df["stemming"].apply(lambda x : word_tokenize(str(x)))
required_df["lemmitization"]=required_df['tokenisation1'].apply(lambda x: ' '.join([textblob.Word(i).lemmatize() for i in x]))

In [21]:
texts = required_df[["lower_case"]]
texts

,lower_case
0,when i saw magnificent heath ledger years go e...
1,i say im really grateful movies like one haven...
2,im fan batman movies went see teenage son its ...
3,what master piece wondeful movie great constru...
4,has best movie ive seen years it feel taxi dri...
...,...
95,absolutely great performance phoenix owns role...
96,one best movies i really recommend gys
97,
98,this movie truly masterpiece the acting cenima...


In [22]:
!pip install pyLDAvis

In [23]:
!pip install biterm

In [30]:
import numpy as np
import pyLDAvis
from biterm.btm import oBTM 
from sklearn.feature_extraction.text import CountVectorizer
from biterm.utility import vec_to_biterms, topic_summuary # helper functions

if __name__ == "__main__":

    # vectorize texts
    vec = CountVectorizer(stop_words='english')
    X = vec.fit_transform(texts["lower_case"].values.tolist()[:50]).toarray()

    # get vocabulary
    vocab = np.array(vec.get_feature_names())
    # get biterms
    biterms = vec_to_biterms(X)

    # create btm
    btm = oBTM(num_topics=20, V=vocab)

    print("\n\n Train Online BTM ..")
    for i in range(0, len(biterms), 100): # prozess chunk of 200 texts
        biterms_chunk = biterms[i:i + 100]
        btm.fit(biterms_chunk, iterations=50)
    topics = btm.transform(biterms)
    
    # print("\n\n Visualize Topics ..")
    # vis = pyLDAvis.prepare(btm.phi_wz.T, topics, np.count_nonzero(X, axis=1), vocab, np.sum(X, axis=0)[:None])
    # pyLDAvis.save_html(vis, '/content/online_btm.html')  # path to output

    print("\n\n Topic coherence ..")
    topic_summuary(btm.phi_wz.T, X, vocab, 10)




 Train Online BTM ..


100%|██████████| 50/50 [05:49<00:00,  6.98s/it]
/usr/local/lib/python3.6/dist-packages/biterm/btm.py:76: RuntimeWarning: invalid value encountered in true_divide
  P_zd[i] = P_zb.sum(axis=0) / P_zb.sum(axis=0).sum()




 Topic coherence ..
Topic 0 | Coherence=-31.55 | Top words= seen truly watch masterpiece society best mental illness like film
Topic 1 | Coherence=-28.99 | Top words= illness mental story person make makes thats batman incredible struggles
Topic 2 | Coherence=-40.62 | Top words= phoenix movie best joker dark todd superb life film joaquin
Topic 3 | Coherence=21.49 | Top words= early affected nominate wild raw social smile fleckin depressed life
Topic 4 | Coherence=-45.20 | Top words= movie watch like thing really know im dont masterpiece went
Topic 5 | Coherence=-45.94 | Top words= movie seen joaquin know twisted watched joker movies past man
Topic 6 | Coherence=-53.31 | Top words= movie best years acting ive seen work mental film rise
Topic 7 | Coherence=-29.23 | Top words= movie performance iconic piercing classicand transformation moments drawn thats focused
Topic 8 | Coherence=-42.60 | Top words= best design incredible film phoenix great writing oscars hes phenomenal
Topic 9 | Coh

In [33]:
topics_data = texts["lower_case"].values.tolist()

In [37]:
    print("\n\n Texts & Topics ..")
    for i in range(len(topics)):
      print("{} (topic: {})".format(topics_data[i], topics[i].argmax()))



 Texts & Topics ..
when i saw magnificent heath ledger years go excellent role joker i doubted one make amazing performance heath years heath tragic death break hearts joaquin phoenix bring tears eyes masterpiece i cant really find words describe phoenix great cinematography absolutely amazing music outstanding i know phoenix great actor especially beautiful role her actually new role ajoker i think put new standard i doubt years one could reach (topic: 17)
i say im really grateful movies like one havent died yet i waited long movie wishing bad one i went theater much hope reviews i finished movie completely silence really thought see even though i never really kind feeling since interstellar its masterpiece everything excellent done passion thanks joaquin and tod movie im speechless its pure drama friends dont miss go watch put happy face silent one could say (topic: 4)
im fan batman movies went see teenage son its truly one best movies i ever seen i never felt movie perfect movie t